In [19]:
import cv2
import numpy as np
import math
import mediapipe as mp
import pulsectl 

In [20]:
# Inicialização do PulseAudio
try:
    pulse = pulsectl.Pulse('volume-control')
except Exception as e:
    print(f"Erro ao inicializar PulseAudio: {e}")
    exit(1)

In [21]:
# Configurações da câmera
wCam, hCam = 640, 480

In [22]:
# Inicializar a captura de vídeo
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Erro ao abrir a câmera.")
    exit(1)

In [23]:
cap.set(cv2.CAP_PROP_FRAME_WIDTH, wCam)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, hCam)

True

In [ ]:
# Mediapipe Hands detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)


In [25]:
# Mediapipe Drawing
mp_draw = mp.solutions.drawing_utils


In [ ]:
try:
    # Loop principal
    while True:
        success, img = cap.read()
        if not success:
            print("Falha ao capturar imagem da câmera.")
            break

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                thumb_tip_coords = (int(thumb_tip.x * wCam), int(thumb_tip.y * hCam))
                index_tip_coords = (int(index_tip.x * wCam), int(index_tip.y * hCam))

                cv2.circle(img, thumb_tip_coords, 15, (255, 0, 0), cv2.FILLED)
                cv2.circle(img, index_tip_coords, 15, (0, 255, 0), cv2.FILLED)
                cv2.line(img, thumb_tip_coords, index_tip_coords, (0, 0, 255), 3)

                distance = math.hypot(index_tip.x - thumb_tip.x, index_tip.y - thumb_tip.y)

                # Ajuste estes valores conforme necessário
                volume_level = np.interp(distance, [0.01, 0.4], [0.0, 1.0])

                print(f"Distance: {distance}, Volume Level: {volume_level}")

                # Ajustar volume aqui
                try:
                    sink = pulse.sink_list()[0]
                    pulse.volume_set_all_chans(sink, volume_level)
                except Exception as e:
                    print(f"Erro ao ajustar volume: {e}")

        cv2.imshow("Hand Tracking", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    pulse.close()  # Garante que a conexão com o PulseAudio seja fechada corretamente